# Simple Custom Model

### Importing the Libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator


In [2]:
tf.__version__

'2.13.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

Mounting Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Btech/dataSet.zip

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [7]:
training_set = train_datagen.flow_from_directory('/content/dataSet/trainingData',
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',
                                                 class_mode = 'categorical')

Found 12845 images belonging to 27 classes.


In [8]:
test_set = test_datagen.flow_from_directory('/content/dataSet/testingData',
                                            target_size = (128, 128),
                                            batch_size = 10,
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 4268 images belonging to 27 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [9]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [10]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3,
                                     padding="same",
                                     activation="relu",
                                     input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [11]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2,
                                         strides=2,
                                         padding='valid'))

#### Adding a second convolutional layer

In [12]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                      kernel_size=3,
                                      padding="same",
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2,
                                         strides=2,
                                         padding='valid'))

#### Step 3 - Flattening

In [13]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [14]:
classifier.add(tf.keras.layers.Dense(units=128,
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [15]:
classifier.compile(optimizer = 'adam',
                   loss = 'categorical_crossentropy',
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [16]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 128)               4

In [19]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model.h5",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [20]:
classifier.fit(training_set,
                  epochs = 30, batch_size = 32,
                  validation_data = test_set,callbacks = [model_checkpoint_callback])

Epoch 1/30
1284/1285 [============================>.] - ETA: 0s - loss: 0.3139 - accuracy: 0.8968
Epoch 1: val_accuracy improved from -inf to 0.98899, saving model to model.h5
1285/1285 [==============================] - 42s 32ms/step - loss: 0.3137 - accuracy: 0.8969 - val_loss: 0.0351 - val_accuracy: 0.9890
Epoch 2/30
1284/1285 [============================>.] - ETA: 0s - loss: 0.2797 - accuracy: 0.9088
Epoch 2: val_accuracy improved from 0.98899 to 0.99367, saving model to model.h5
1285/1285 [==============================] - 41s 32ms/step - loss: 0.2798 - accuracy: 0.9087 - val_loss: 0.0194 - val_accuracy: 0.9937
Epoch 3/30
1283/1285 [============================>.] - ETA: 0s - loss: 0.2391 - accuracy: 0.9251
Epoch 3: val_accuracy improved from 0.99367 to 0.99485, saving model to model.h5
1285/1285 [==============================] - 40s 31ms/step - loss: 0.2390 - accuracy: 0.9251 - val_loss: 0.0192 - val_accuracy: 0.9948
Epoch 4/30
1284/1285 [============================>.] - ETA: 

#### Saving the Model

In [21]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new.h5')
print('Weights saved')

Model Saved
Weights saved
